In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
cat_dir = "PetImages/Cat"
dog_dir = "PetImages/Dog"
# Image size
IMG_SIZE = 128

In [ ]:
# Load images and labels into memory
def load_images(folder, label):
    images = []
    labels = []
    count = 1
    # x = os.listdir(folder)
    for filename in os.listdir(folder):
        
        if filename.lower().endswith(('jpg', 'jpeg', 'png')):
            try:
                path = os.path.join(folder, filename)
                img = Image.open(path).convert("RGB")
                img = img.resize((IMG_SIZE, IMG_SIZE))
                img = np.array(img) / 255.0  # Normalize
                images.append(img)
                labels.append(label)
                count+=1
                if count == 10:
                    break
            except:
                continue  # Skip corrupted files
    return images, labels


In [ ]:
cat_images, cat_labels = load_images(cat_dir, 0)
dog_images, dog_labels = load_images(dog_dir, 1)

In [ ]:

# Combine and split
X = np.array(cat_images + dog_images)
y = np.array(cat_labels + dog_labels)

# One-hot encode labels
y = tf.keras.utils.to_categorical(y, num_classes=2)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation = 'relu', input_shape=(IMG_SIZE,IMG_SIZE,3)),
    layers.MaxPooling2D(4,4),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [ ]:
# Train
history = model.fit(X_train, y_train, validation_split=0.2, steps_per_epoch=2, epochs=2, batch_size=8)

In [ ]:
history.history


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=32)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Accuracy vs Epochs")
plt.show()

# Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Loss vs Epochs")
plt.show()


In [ ]:
model.save("cat_dog_classifier.h5")  # Saves in HDF5 format
# from tensorflow.keras.models import load_model
# model = load_model("cat_dog_classifier.h5")